In [ ]:
%matplotlib widget 
# Si se quiere usar %matplotlib qt hay que instalar pip install PyQt6
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.animation as animation

# Animación con FuncAnimation:

``class matplotlib.animation.FuncAnimation(fig, func, frames=None, init_func=None, fargs=None, save_count=None, *, cache_frame_data=True, **kwargs)[source]
``

In [ ]:
fig, ax = plt.subplots()
t = np.linspace(0, 3, 40)
g = -9.81
v0 = 12
z = g * t**2 / 2 + v0 * t

v02 = 5
z2 = g * t**2 / 2 + v02 * t

scat = ax.scatter(t[0], z[0], c="b", s=5, label=f'v0 = {v0} m/s')
line2 = ax.plot(t[0], z2[0], label=f'v0 = {v02} m/s')[0]
ax.set(xlim=[0, 3], ylim=[-4, 10], xlabel='Time [s]', ylabel='Z [m]')
ax.legend()


def update(frame):
    # for each frame, update the data stored on each artist.
    x = t[:frame]
    y = z[:frame]
    # update the scatter plot:
    data = np.stack([x, y]).T
    scat.set_offsets(data)
    # update the line plot:
    line2.set_xdata(t[:frame])
    line2.set_ydata(z2[:frame])
    return (scat, line2)


ani = animation.FuncAnimation(fig=fig, func=update, frames=40, interval=30)
plt.show()

## Animación con ArtistAnimation

``class matplotlib.animation.ArtistAnimation(fig, artists, *args, **kwargs)``

In [ ]:
fig, ax = plt.subplots()
rng = np.random.default_rng(19680801)
data = np.array([20, 20, 20, 20])
x = np.array([1, 2, 3, 4])

artists = []
colors = ['tab:blue', 'tab:red', 'tab:green', 'tab:purple']

for i in range(20):
    data += rng.integers(low=0, high=10, size=data.shape)
    container = ax.barh(x, data, color=colors)
    artists.append(container)

ani = animation.ArtistAnimation(fig=fig, artists=artists, interval=400)
plt.show()

# Plantilla
### Grafico estilo grilla pero en ipywidget
Se inserta todo el código de matplotlib en una función, exceptuando las variables aunque podrian estar dentro de la función, pero es común dejarlas afuera.

#### Se agregan:

``import ipywidgets as widgets``

``from IPython.display import display``

#### Ademas se realizan las siguientes diferencias:
- ``fig = plt.figure(figsize=(12, 12), dpi=200, facecolor='w', edgecolor='k')`` pasa a ``fig, ax = plt.subplots(figsize=(12, 12), dpi=200, facecolor='w', edgecolor='k')`` y se elimina ``ax = plt.gca()``
- ``fig.canvas.draw()`` pasa a ``ax.plot(x, y)``
- como los sliders x_max e y_max se crean como widgets se cambian los limites(bound)
- #### Crear sliders para ajustar los límites de los ejes
    - ``x_max_slider = widgets.IntSlider(value=10, min=1, max=20, description='x max')``
    - ``y_max_slider = widgets.IntSlider(value=10, min=1, max=20, description='y max')``
- #### Llamar a la función de actualización cuando se cambian los valores de los sliders
    - ``widgets.interactive(update_plot, x_max=x_max_slider, y_max=y_max_slider)``

In [2]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
import numpy as np
import ipywidgets as widgets
from IPython.display import display

_fontsize_legend = 10
_fontsize = 15
DP = 3

# Datos a graficar: #

def update_plot(x_max, y_max):
    #plt.clf()  # Limpiar la figura actual
    #fig = plt.figure(figsize=(12, 12), dpi=200, facecolor='w', edgecolor='k')
    fig, ax = plt.subplots(figsize=(12, 12), dpi=200, facecolor='w', edgecolor='k') #ipywidget crea figure y axes en una linea y usa subplots()
    #fig.canvas.draw() # ax.plot(x, y) es igual pero hace que se grafique en ipywidget
    ax = plt.gca() #Desactivar cuando se tengan datos y usar ax.plot(x,y)
    #ax.plot(x,y) # Con ipywidget es necesario!
    ax.spines['left'].set_position('zero') # Eje izquierdo que pasa por cero (haciendo de eje y)
    ax.spines['right'].set_color('none') # color del eje derecho se pone none
    ax.spines['bottom'].set_position('zero') # Eje inferior que pasa por cero (haciendo de eje x)
    ax.spines['top'].set_color('none') # color del eje superior se pone none
    ax.xaxis.set_ticks_position('bottom') # Marcas de graduación debajo del eje x
    ax.yaxis.set_ticks_position('left') # Marcas de graduación a la izqquierda del eje y
    
    ax.set_ybound(-1,y_max) # ax.set_xlim(-1, x_max) es igual, para aprovechar ipywidgets se cambio 10 por y_max
    ax.set_xbound(-1,x_max) # ax.set_ylim(-1, y_max) es igual
    
    ax.xaxis.set_major_locator(MultipleLocator(1)) # ax.xaxis.set_major_locator(plt.MultipleLocator(1)) con ipywidgets difiere por que no se importo MultipleLocator entonces se lo llama con plt.Multiplicator
    ax.yaxis.set_major_locator(MultipleLocator(1))
    
    ax.xaxis.grid(True,'major',linewidth=2/DP,linestyle='-',color='#d7d7d7',zorder=0) # ax.xaxis.grid(True, 'major', linewidth=2/3, linestyle='-', color='#d7d7d7', zorder=0) es igual no usa la variable DP con ipywidget
    ax.yaxis.grid(True,'major',linewidth=2/DP,linestyle='-',color='#d7d7d7')
    ax.xaxis.set_minor_locator(MultipleLocator( (1) / 10 ))
    ax.yaxis.set_minor_locator(MultipleLocator( (1) / 10 ))
    ax.xaxis.grid(True,'minor',linewidth=0.5/DP,linestyle='-',color='#d7d7d7') # Verticales cada 1 division(mm) en el eje x
    # ax.xaxis.grid(True, 'minor', linewidth=0.5/3, linestyle='-', color='#d7d7d7', zorder=0) con ipywidgets no usa DP y agrega zorder 
    ax.yaxis.grid(True,'minor',linewidth=0.5/DP,linestyle='-',color='#d7d7d7') # horizontales cada 1 división(mm) en el eje y
    
    ax.set_axisbelow(True) # Mandar al fondo a las grillas
    ax.set_aspect('equal') # Asegura que la distancia entre los ticks y la   grafica se mantengan iguales en ambos ejes
    # ax.xaxis.set_major_formatter(plt.FormatStrFormatter('%i')) con ipywidgets 
    ax.xaxis.set_major_formatter(FormatStrFormatter('%i')) # Formateador entero para eje x
    xticks = ax.xaxis.get_major_ticks() # Obtener la lista de ticks en eje x para poder iterar con for
    for i,l in enumerate(xticks):
        if not (i - 1) % 4 == 0: # Hace que el numero pueda ser visible cada 4
            xticks[i].label1.set_visible(True) # True para que muestre cada digito
        else:
            xticks[i].label1.set_fontsize(_fontsize)
    
    ax.yaxis.set_major_formatter(FormatStrFormatter('%i')) # Formateador entero para eje y
    yticks = ax.yaxis.get_major_ticks() # Obtener la lista de ticks en eje y para poder iterar con for
    for i,l in enumerate(yticks):
        if not (i - 1) % 4 == 0:
            yticks[i].label1.set_visible(True)
        else:
            yticks[i].label1.set_fontsize(_fontsize)

# Crear sliders para ajustar los límites de los ejes
x_max_slider = widgets.IntSlider(value=10, min=1, max=20, description='x max')
y_max_slider = widgets.IntSlider(value=10, min=1, max=20, description='y max')


# Llamar a la función de actualización cuando se cambian los valores de los sliders
widgets.interactive(update_plot, x_max=x_max_slider, y_max=y_max_slider)

interactive(children=(IntSlider(value=10, description='x max', max=20, min=1), IntSlider(value=10, description…